In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("Pexample").getOrCreate()

In [3]:
data = [
    ('2024-10-05 12:34:56', 'param1', 10),
    ('2024-10-05 12:45:00', 'param1', 20),
    ('2024-10-05 13:05:00', 'param2', 30),
    ('2024-10-05 13:30:00', 'param1', 40),
]

In [6]:
df = spark.createDataFrame(data,['time','param','value'])
df.show()

+-------------------+------+-----+
|               time| param|value|
+-------------------+------+-----+
|2024-10-05 12:34:56|param1|   10|
|2024-10-05 12:45:00|param1|   20|
|2024-10-05 13:05:00|param2|   30|
|2024-10-05 13:30:00|param1|   40|
+-------------------+------+-----+



In [7]:
df.withColumn('time',F.to_timestamp(F.col('time')))
df.printSchema()

root
 |-- time: string (nullable = true)
 |-- param: string (nullable = true)
 |-- value: long (nullable = true)



In [8]:
df.show()

+-------------------+------+-----+
|               time| param|value|
+-------------------+------+-----+
|2024-10-05 12:34:56|param1|   10|
|2024-10-05 12:45:00|param1|   20|
|2024-10-05 13:05:00|param2|   30|
|2024-10-05 13:30:00|param1|   40|
+-------------------+------+-----+



In [11]:
df.groupBy(F.date_trunc('hour',F.col("time")).alias('time_hours'), 'param').agg(F.avg(F.col('value')).alias("Avg_value")).show()

+-------------------+------+---------+
|         time_hours| param|Avg_value|
+-------------------+------+---------+
|2024-10-05 12:00:00|param1|     15.0|
|2024-10-05 13:00:00|param2|     30.0|
|2024-10-05 13:00:00|param1|     40.0|
+-------------------+------+---------+



In [13]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [14]:
schema = StructType([
    StructField("emp_id", IntegerType(), True),       # Employee ID
    StructField("emp_name", StringType(), True),     # Employee Name
    StructField("emp_department", StringType(), True), # Employee Department
    StructField("emp_salary", FloatType(), True)      # Employee Salary
])

In [15]:
data = [
    # Engineering Department
    (1, "Alice Johnson", "Engineering", 75000.0),
    (2, "Bob Smith", "Engineering", 80000.0),
    (3, "Charlie Brown", "Engineering", 72000.0),
    (4, "David Wilson", "Engineering", 68000.0),
    (5, "Eva Green", "Engineering", 90000.0),
    (6, "Frank Wright", "Engineering", 85000.0),
    
    # HR Department
    (7, "Gina Roberts", "HR", 60000.0),
    (8, "Henry Ford", "HR", 62000.0),
    (9, "Ivy Adams", "HR", 58000.0),
    (10, "Jack Nelson", "HR", 65000.0),
    (11, "Kathy Lee", "HR", 59000.0),
    (12, "Laura King", "HR", 61000.0),
    
    # Finance Department
    (13, "Michael Scott", "Finance", 80000.0),
    (14, "Nina Patel", "Finance", 82000.0),
    (15, "Oscar Martinez", "Finance", 78000.0),
    (16, "Pam Beesly", "Finance", 77000.0),
    (17, "Quinn Kim", "Finance", 79000.0),
    (18, "Rita Thomas", "Finance", 81000.0),

    # Marketing Department
    (19, "Sam Brown", "Marketing", 65000.0),
    (20, "Tina Turner", "Marketing", 68000.0),
    (21, "Ursula White", "Marketing", 67000.0),
    (22, "Victor Hugo", "Marketing", 66000.0),
    (23, "Wendy Lee", "Marketing", 69000.0),
    (24, "Xena Gold", "Marketing", 72000.0)
]

In [16]:
df = spark.createDataFrame(data,schema)
df.show()

+------+--------------+--------------+----------+
|emp_id|      emp_name|emp_department|emp_salary|
+------+--------------+--------------+----------+
|     1| Alice Johnson|   Engineering|   75000.0|
|     2|     Bob Smith|   Engineering|   80000.0|
|     3| Charlie Brown|   Engineering|   72000.0|
|     4|  David Wilson|   Engineering|   68000.0|
|     5|     Eva Green|   Engineering|   90000.0|
|     6|  Frank Wright|   Engineering|   85000.0|
|     7|  Gina Roberts|            HR|   60000.0|
|     8|    Henry Ford|            HR|   62000.0|
|     9|     Ivy Adams|            HR|   58000.0|
|    10|   Jack Nelson|            HR|   65000.0|
|    11|     Kathy Lee|            HR|   59000.0|
|    12|    Laura King|            HR|   61000.0|
|    13| Michael Scott|       Finance|   80000.0|
|    14|    Nina Patel|       Finance|   82000.0|
|    15|Oscar Martinez|       Finance|   78000.0|
|    16|    Pam Beesly|       Finance|   77000.0|
|    17|     Quinn Kim|       Finance|   79000.0|


In [19]:
#find third highest salary

third_highest_sal = df.orderBy('emp_salary',ascending=False).limit(3).tail(1)

In [21]:
third_highest_sal

[Row(emp_id=14, emp_name='Nina Patel', emp_department='Finance', emp_salary=82000.0)]

In [22]:
#Second highest salary in each department

from pyspark.sql import Window

In [24]:
window = Window.partitionBy(F.col('emp_department')).orderBy(F.col('emp_salary').desc())

In [26]:
df_rank = df.withColumn('rank',F.dense_rank().over(window))
df_rank.show()

+------+--------------+--------------+----------+----+
|emp_id|      emp_name|emp_department|emp_salary|rank|
+------+--------------+--------------+----------+----+
|     5|     Eva Green|   Engineering|   90000.0|   1|
|     6|  Frank Wright|   Engineering|   85000.0|   2|
|     2|     Bob Smith|   Engineering|   80000.0|   3|
|     1| Alice Johnson|   Engineering|   75000.0|   4|
|     3| Charlie Brown|   Engineering|   72000.0|   5|
|     4|  David Wilson|   Engineering|   68000.0|   6|
|    14|    Nina Patel|       Finance|   82000.0|   1|
|    18|   Rita Thomas|       Finance|   81000.0|   2|
|    13| Michael Scott|       Finance|   80000.0|   3|
|    17|     Quinn Kim|       Finance|   79000.0|   4|
|    15|Oscar Martinez|       Finance|   78000.0|   5|
|    16|    Pam Beesly|       Finance|   77000.0|   6|
|    10|   Jack Nelson|            HR|   65000.0|   1|
|     8|    Henry Ford|            HR|   62000.0|   2|
|    12|    Laura King|            HR|   61000.0|   3|
|     7|  

In [28]:
df_rank.filter(F.col('rank')==3).show()

+------+-------------+--------------+----------+----+
|emp_id|     emp_name|emp_department|emp_salary|rank|
+------+-------------+--------------+----------+----+
|     2|    Bob Smith|   Engineering|   80000.0|   3|
|    13|Michael Scott|       Finance|   80000.0|   3|
|    12|   Laura King|            HR|   61000.0|   3|
|    20|  Tina Turner|     Marketing|   68000.0|   3|
+------+-------------+--------------+----------+----+



In [32]:
def di(x):
    return {i:(i**2 if i%2==0 else i**3) for i in range(1,x+1)}
print(di(10))

{1: 1, 2: 4, 3: 27, 4: 16, 5: 125, 6: 36, 7: 343, 8: 64, 9: 729, 10: 100}


In [33]:
repeated_values =[2,3,4,5,6,6,7,8,8,9,9,9,4,4]
#get the values which are repeated more than once

from collections import Counter
counts = Counter(repeated_values)
print(counts)

Counter({4: 3, 9: 3, 6: 2, 8: 2, 2: 1, 3: 1, 5: 1, 7: 1})


In [34]:
new_lst = []
for key,value in counts.items():
    if value>1:
        new_lst.append(key)
    else:
        continue

In [35]:
new_lst

[4, 6, 8, 9]

In [36]:
seen = set()
repeated = set()

for ele in repeated_values:
    if ele in seen:
        repeated.add(ele)
    else:
        seen.add(ele)
print(list(repeated))

[8, 9, 4, 6]


In [37]:
import pandas as pd
from datetime import datetime

# Your data
data = {
    'Employee_ID': [101, 102, 103],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Joining_Date': ['2017-05-10', '2015-06-25', '2020-08-15'],
    'Salary': [60000, 75000, 50000]
}

# Convert to DataFrame
df = pd.DataFrame(data)

In [38]:

df

,Employee_ID,Name,Joining_Date,Salary
0,101,Alice,2017-05-10,60000
1,102,Bob,2015-06-25,75000
2,103,Charlie,2020-08-15,50000


In [39]:
df['Joining_Date'] = pd.to_datetime(df['Joining_Date'])

In [40]:
df

,Employee_ID,Name,Joining_Date,Salary
0,101,Alice,2017-05-10,60000
1,102,Bob,2015-06-25,75000
2,103,Charlie,2020-08-15,50000


In [41]:
df['experience'] = (datetime.now()-df['Joining_Date']).dt.days//365

In [42]:
df

,Employee_ID,Name,Joining_Date,Salary,experience
0,101,Alice,2017-05-10,60000,7
1,102,Bob,2015-06-25,75000,9
2,103,Charlie,2020-08-15,50000,4


In [43]:
def update_salary(row):
    if row['experience'] > 10:
        return row['Salary']+10000
    elif row['experience'] >5:
        return row['Salary']+5000
    else:
        return row['Salary']
df['Salary'] = df.apply(update_salary,axis=1)

In [44]:
df

,Employee_ID,Name,Joining_Date,Salary,experience
0,101,Alice,2017-05-10,65000,7
1,102,Bob,2015-06-25,80000,9
2,103,Charlie,2020-08-15,50000,4
